In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install -U pip
!pip install -U setuptools wheel
!pip install -U "autogluon.timeseries[all]"
!pip install -U transformers accelerate

  Using cached autogluon_timeseries-1.5.0-py3-none-any.whl.metadata (13 kB)
  Using cached lightning-2.5.6-py3-none-any.whl.metadata (42 kB)
  Using cached gluonts-0.16.2-py3-none-any.whl.metadata (9.8 kB)
  Using cached statsforecast-2.0.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (29 kB)
  Using cached mlforecast-0.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached utilsforecast-0.2.11-py3-none-any.whl.metadata (7.7 kB)
  Using cached coreforecast-0.0.16-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.7 kB)
  Using cached fugue-0.9.4-py3-none-any.whl.metadata (18 kB)
  Using cached chronos_forecasting-2.2.2-py3-none-any.whl.metadata (23 kB)
  Using cached peft-0.17.1-py3-none-any.whl.metadata (14 kB)
  Using cached autogluon_core-1.5.0-py3-none-any.whl.metadata (13 kB)
  Using cached autogluon_common-1.5.0-py3-none-any.whl.metadata (12 kB)
  Using cached autogluon_features-1.5.0-py3-none-any.whl.metadata (12 kB)
  Using cached autog

In [4]:
import pandas as pd
import torch
import numpy as np
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor
import matplotlib.pyplot as plt
import os

In [5]:
COVARIATE_COLUMNS = ['time_hour_sin', 'time_hour_cos', 'time_dayofyear_sin']

In [6]:
USE_CV2_SAVED_MODEL=False

In [2]:
def load_and_prepare(path):
    print(f"Loading {path}...")
    df = pd.read_parquet(path)

    # 1. Inspect initial columns (for debugging)
    print(f"Original columns: {list(df.columns)}")

    # 2. Rename columns to match AutoGluon's expected format
    # Map 'time' -> 'timestamp' and 'series_id' -> 'item_id'
    column_mapping = {
        "time": "timestamp",
        "series_id": "item_id",
        # If your target is 'pv', we can keep it or rename to 'target'/'pv_value'
        "pv": "pv_value"
    }

    # Only rename columns that actually exist
    df = df.rename(columns={k: v for k, v in column_mapping.items() if k in df.columns})

    # 3. Force Timestamp Conversion & Remove Timezone
    # AutoGluon requires naive datetime64[ns]
    df['timestamp'] = pd.to_datetime(df['timestamp'])

    if df['timestamp'].dt.tz is not None:
        print("Detected timezone info. Removing for AutoGluon compatibility...")
        df['timestamp'] = df['timestamp'].dt.tz_localize(None)

    print(f"Final columns: {list(df.columns)}")
    print(f"Timestamp type: {df['timestamp'].dtype}")

    missing_covariates = [col for col in COVARIATE_COLUMNS if col not in df.columns]
    if missing_covariates:
        raise ValueError(f"Missing required covariate columns: {missing_covariates}")

    # 4. Create TimeSeriesDataFrame
    # Now we are sure 'item_id' and 'timestamp' exist
    ts_df = TimeSeriesDataFrame.from_data_frame(
        df,
        id_column="item_id",
        timestamp_column="timestamp"
    )

    for col in COVARIATE_COLUMNS:
        ts_df[col] = ts_df[col].astype(float)

    print(f"TimeSeriesDataFrame created. Shape: {ts_df.shape}")
    print(f"Included features: {list(ts_df.columns)}")

    return ts_df

In [7]:
def fit_model(full_df, use_saved_models=USE_CV2_SAVED_MODEL):
  train_data = TimeSeriesDataFrame.from_data_frame(
    full_df,
    id_column="item_id",
    timestamp_column="timestamp"
  )

  # Verify frequency and check for internal gaps (should be none within segments)
  print("\nVerifying Data Integrity...")
  train_data = train_data.convert_frequency(freq='30min')
  print(train_data.head())

  # Define prediction horizon (e.g., 2 days = 48 hours = 96 steps)
  PREDICTION_LENGTH = 96
  model_hyperparameters = {
      "Chronos": [
          {
          "model_path": "amazon/chronos-bolt-small",
          "batch_size": 32,
          "context_length": 512,
          "optimization.max_epochs": 10, # Increase for better results
      },
      {
          "model_path": "amazon/chronos-bolt-base",
          "batch_size": 16,
          "context_length": 512,
          "ag_args": {"name_suffix": "ZeroShot"},
          "optimization.max_epochs": 10, # Increase for better results
      },
      {
          "model_path": "amazon/chronos-bolt-base",
          "batch_size": 16,
          "context_length": 512,
          "covariate_regressor": "CAT",
          "target_scaler": "standard",
          "ag_args": {"name_suffix": "WithRegressor"},
          "optimization.max_epochs": 10, # Increase for better results
      }
      ]
  }

  bolt_predictor = TimeSeriesPredictor(
      prediction_length=PREDICTION_LENGTH,
      path="autogluon_chronos_pv_forecast",
      target="pv_value",
      eval_metric="MASE", # Mean Absolute Scaled Error
      #freq='30min',
      known_covariates_names=COVARIATE_COLUMNS
  )

  print("\nStarting Training with Bolt model(s)")
  bolt_predictor.fit(
      train_data,
      hyperparameters=model_hyperparameters,
      enable_ensemble=False, # Disable ensemble to isolate Chronos performance
      random_seed=42
  )

  # IF I USED SAVED MODELS FOR CHRONOS 2
  if use_saved_models:
    return train_data, bolt_predictor, None

  # 3. Initialize the Predictor
  # We must specify 'target="pv_value"' because you renamed the 'pv' column.
  c2_predictor = TimeSeriesPredictor(
      prediction_length=PREDICTION_LENGTH,
      target="pv_value",
      path="autogluon_chronos2_results", # Folder to save models
      eval_metric="MASE", # Mean Absolute Scaled Error (common for forecasting)
      known_covariates_names=COVARIATE_COLUMNS
  )

  robust_hyperparameters = {
    "Chronos2": [  # Use "Chronos" as the key for all Chronos variants (including Chronos-2)
        # 1. Zero-shot version (Baseline)
        {
            "ag_args": {"name_suffix": "ZeroShot"}
        },

        # 2. Fine-tuned version (The one you want to watch)
        {
            "fine_tune": True,
            #"fine_tune_steps": 500,  # Explicit steps so you know how long it runs
            "ag_args": {"name_suffix": "FineTuned"},

            # --- THIS IS THE KEY PART ---
            "fine_tune_trainer_kwargs": {
                "disable_tqdm": False,       # Turn the progress bar back ON
                "logging_steps": 10,         # Log loss every 10 steps
                "report_to": ["tensorboard"] # Optional: Log to tensorboard if installed
            }
        },
    ]
  }

  print("Running Chronos-2 with visible fine-tuning...")
  c2_predictor.fit(
      full_df,
      hyperparameters=robust_hyperparameters,
      num_val_windows=2,
      val_step_size=PREDICTION_LENGTH,
      verbosity=2  # Increase verbosity to 3 or 4 to ensure underlying logs pass through
  )
  """
  robust_hyperparameters = {
    "Chronos2": [
        {"ag_args": {"name_suffix": "ZeroShot"}},
        # Fine-tuned model
        {"fine_tune": True, "ag_args": {"name_suffix": "FineTuned"}},
    ]
  }

  print("Running Chronos-2...")
  c2_predictor.fit(
      full_df,
      hyperparameters=robust_hyperparameters,
      #presets="chronos2",
      num_val_windows=3,              # Perform 3-fold backtesting
      val_step_size=PREDICTION_LENGTH,
      # If you want to fine-tune instead of zero-shot, you can add hyperparameters:
      # hyperparameters={"Chronos": {"fine_tune": True}}
      verbosity=2
  )
  """
  return train_data, bolt_predictor, c2_predictor

In [8]:
def calculate_item_mase(y_true, y_pred, y_history):
    """
    Calculates MASE for a single item.
    MASE = MAE(forecast) / MAE(naive_history)
    """
    # 1. Calculate MAE of the forecast (Numerator)
    # Ensure alignment
    mae_forecast = np.mean(np.abs(y_true.values - y_pred.values))

    # 2. Calculate MAE of naive forecast on history (Denominator)
    # We use lag-1 (standard naive) for the denominator
    if len(y_history) < 2:
        return np.nan # Not enough history

    # Naive error: mean(|t - (t-1)|)
    naive_errors = np.abs(np.diff(y_history.values))
    mae_naive = np.mean(naive_errors)

    if mae_naive == 0:
        return np.inf if mae_forecast > 0 else 0.0

    return mae_forecast / mae_naive

In [9]:
def plot_prediction(past_data, full_data, bolt_predictor, c2_predictions, known_covariates_future):
    """
    past_data: The data used for input (truncated)
    full_data: The original full data (containing the ground truth for evaluation)
    """
    print("\nGenerating Backtest Forecasts (Hiding last steps)...")
    PLOT_LENGTH= 200
    # 1. Generate Bolt predictions
    model_names = bolt_predictor.model_names()
    model_predictions = {}

    for model_name in model_names:
        print(f"Predicting with {model_name}...")
        model_predictions[model_name] = bolt_predictor.predict(past_data, known_covariates=known_covariates_future, model=model_name)

    for c2_name, c2_pred in c2_predictions.items():
        # Append to the list of names so the plotter loops over it
        model_names.append(c2_name)
        # Store the prediction dataframe
        model_predictions[c2_name] = c2_pred

    # 2. Add Chronos-2 predictions
    #for predictio
    #model_names.append("Chronos_2")
    #model_predictions["Chronos_2"] = c2_predictions

    # 3. Setup Plotting
    item_ids = sorted(full_data.reset_index()['item_id'].unique())
    num_plots = len(item_ids)

    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']

    fig, axs = plt.subplots(num_plots, 1, figsize=(15, 6 * num_plots), sharex=False)
    if num_plots == 1: axs = [axs]

    for i, item_id in enumerate(item_ids):
        ax = axs[i]

        # --- A. Prepare Data ---
        # Full history for plotting context
        history_context = full_data.loc[item_id].iloc[-(PLOT_LENGTH + PREDICTION_LENGTH):]

        # Ground Truth (The actual future we are trying to predict)
        # It corresponds to the last PREDICTION_LENGTH steps of the full data
        ground_truth_future = full_data.loc[item_id].iloc[-PREDICTION_LENGTH:]['pv_value']

        # History used for MASE denominator (everything BEFORE the prediction window)
        # Note: We take enough history to get a stable naive error, e.g., all available history or last 500 steps
        history_for_metric = full_data.loc[item_id].iloc[:-PREDICTION_LENGTH]['pv_value']

        # --- B. Plot Ground Truth ---
        ax.plot(history_context.index, history_context['pv_value'],
                label='Actual Ground Truth', color='black', linewidth=2, alpha=0.6)

        # Highlight the Forecast Window
        cutoff_date = ground_truth_future.index[0]
        ax.axvspan(cutoff_date, ground_truth_future.index[-1], color='gray', alpha=0.1, label="Forecast Window")

        # --- C. Plot Each Model & Calculate MASE ---
        for idx, model_name in enumerate(model_names):
            preds = model_predictions[model_name]
            forecast = preds.loc[item_id]

            # 1. Calculate MASE
            # Align forecast with ground truth (just to be safe, though indices should match)
            # We assume forecast index matches ground_truth_future index
            try:
                mase_score = calculate_item_mase(
                    y_true=ground_truth_future,
                    y_pred=forecast['mean'],
                    y_history=history_for_metric
                )
                mase_label = f"MASE: {mase_score:.3f}"
            except Exception as e:
                print(f"Error calc MASE for {model_name}: {e}")
                mase_label = "MASE: N/A"

            # 2. Prepare Label
            clean_name = model_name.split('/')[-1]
            label_text = f'{clean_name} | {mase_label}'
            color = colors[idx % len(colors)]

            # 3. Plot Mean
            ax.plot(forecast.index, forecast['mean'],
                    label=label_text, color=color, linewidth=2, linestyle='--')

            # 4. Plot Confidence Interval
            if '0.1' in forecast.columns and '0.9' in forecast.columns:
                ax.fill_between(
                    forecast.index,
                    forecast['0.1'],
                    forecast['0.9'],
                    color=color, alpha=0.15
                )

        # Formatting
        ax.set_title(f"Segment {item_id}: Backtest Performance", fontsize=14, fontweight='bold')
        ax.set_ylabel("PV Value", fontsize=10)
        ax.axvline(x=cutoff_date, color='red', linestyle=':', linewidth=1.5, label="Prediction Start")
        ax.grid(True, which='both', alpha=0.3)
        ax.legend(loc='upper left', fontsize=9, framealpha=0.9)

    plt.tight_layout()
    plt.show()

In [10]:
base_dir = DATA_PATH = "/content/drive/MyDrive/FM_project/dataset"
#train_path = os.path.join(base_dir, "skippd_train_cleaned_30min_no_images_v12.parquet")
train_path = os.path.join(base_dir, "skippd_train_aligned_v13_with_time_features_no_images.parquet")
PREDICTION_LENGTH = 96

In [11]:
def chronos2prediction(past_data, known_covariates_future, known_covariates, c2_predictor, use_saved_models=True):
  if not use_saved_models:
    cv2_model_names = c2_predictor.model_names()
    cv2_model_predictions = {}

    for model_name in cv2_model_names:
        print(f"Predicting with {model_name}...")
        cv2_model_predictions[model_name] = c2_predictor.predict(past_data, known_covariates=known_covariates_future, model=model_name)
    return cv2_model_predictions


  predictor = TimeSeriesPredictor.load("restored_right/autogluon_chronos2_results")
  target_models = [
      "Chronos2FineTuned",
      "Chronos2ZeroShot",
      "WeightedEnsemble"
  ]
  predictions_dict = {}

  print("Available models in this predictor:", predictor.model_names())

  for model_name in target_models:
      if model_name in predictor.model_names():
          print(f"Generating forecast for: {model_name}...")

          preds = predictor.predict(
              past_data,
              known_covariates=known_covariates_future,
              model=model_name
          )

          predictions_dict[model_name] = preds

          # Optional: Save each to CSV
          preds.to_csv(f"forecast_{model_name}.csv")
      else:
          print(f"Warning: Model '{model_name}' not found in predictor.")

  return predictions_dict

In [12]:
full_df = load_and_prepare(train_path)

print(f"Data shape: {full_df.shape}")
print(f"Unique Item IDs (Segments): {full_df.item_ids}")

# Fit models (Returns full trained data for context)
if not USE_CV2_SAVED_MODEL:
  train_data, bolt_predictor, c2_predictor = fit_model(full_df, use_saved_models=USE_CV2_SAVED_MODEL)
else:
  train_data, bolt_predictor, c2_predictor = fit_model(full_df, use_saved_models=USE_CV2_SAVED_MODEL)


# --- Generate Predictions ---
# 1. Create a "Past" dataset by slicing off the last PREDICTION_LENGTH steps
#    The model sees data up to (Now - 96 steps)
past_data = train_data.slice_by_timestep(None, -PREDICTION_LENGTH)
known_covariates_future = train_data[COVARIATE_COLUMNS]


# 2. Predict with Chronos-2 manually (Bolt is handled inside plot function loop)
print("Generating Chronos-2 predictions...")
cv2_model_predictions=chronos2prediction(past_data, known_covariates_future,known_covariates=known_covariates_future, c2_predictor=c2_predictor, use_saved_models=USE_CV2_SAVED_MODEL)

plot_prediction(past_data, train_data, bolt_predictor, cv2_model_predictions, known_covariates_future)

"""
if not USE_CV2_SAVED_MODEL:
  cv2_model_names = c2_predictor.model_names()
  cv2_model_predictions = {}

  for model_name in cv2_model_names:
      print(f"Predicting with {model_name}...")
      cv2_model_predictions[model_name] = c2_predictor.predict(past_data, known_covariates=known_covariates_future, model=model_name)
else:
  cv2_model_predictions=chronos2prediction(past_data, known_covariates_future, use_saved_models=USE_CV2_SAVED_MODEL)

# 3. Plot with MASE
# Important: We pass 'train_data' (the full dataset) so we can compare predictions vs actuals
plot_prediction(past_data, train_data, bolt_predictor, cv2_model_predictions, known_covariates_future)
"""

Loading /content/drive/MyDrive/FM_project/dataset/skippd_train_aligned_v13_with_time_features_no_images.parquet...
Original columns: ['time', 'pv', 'series_id', 'time_hour_sin', 'time_hour_cos', 'time_dayofyear_sin']
Detected timezone info. Removing for AutoGluon compatibility...
Final columns: ['timestamp', 'pv_value', 'item_id', 'time_hour_sin', 'time_hour_cos', 'time_dayofyear_sin']
Timestamp type: datetime64[ns]
TimeSeriesDataFrame created. Shape: (18667, 4)
Included features: ['pv_value', 'time_hour_sin', 'time_hour_cos', 'time_dayofyear_sin']
Data shape: (18667, 4)
Unique Item IDs (Segments): Index([0, 1, 2], dtype='int64', name='item_id')

Verifying Data Integrity...


Beginning AutoGluon training...
AutoGluon will save models to '/content/autogluon_chronos_pv_forecast'
=================== System Info ===================
AutoGluon Version:  1.5.0
Python Version:     3.12.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu Oct  2 10:42:05 UTC 2025
CPU Count:          2
Pytorch Version:    2.9.0+cu126
CUDA Version:       12.6
GPU Memory:         GPU 0: 14.74/14.74 GB
Total GPU Memory:   Free: 14.74 GB, Allocated: 0.00 GB, Total: 14.74 GB
GPU Count:          1
Memory Avail:       10.74 GB / 12.67 GB (84.7%)
Disk Space Avail:   73.08 GB / 112.64 GB (64.9%)

Fitting with arguments:
{'enable_ensemble': False,
 'eval_metric': MASE,
 'hyperparameters': {'Chronos': [{'batch_size': 32,
                                  'context_length': 512,
                                  'model_path': 'amazon/chronos-bolt-small',
                                  'optimization.max_epochs': 10},
                                 {'ag_args':

                             pv_value  time_hour_sin  time_hour_cos  \
item_id timestamp                                                     
0       2017-05-08 07:30:00  0.000000          0.966         -0.259   
        2017-05-08 08:00:00  0.286534          0.866         -0.500   
        2017-05-08 08:30:00  0.769674          0.866         -0.500   
        2017-05-08 09:00:00  2.747408          0.707         -0.707   
        2017-05-08 09:30:00  6.135439          0.707         -0.707   

                             time_dayofyear_sin  
item_id timestamp                                
0       2017-05-08 07:30:00                0.81  
        2017-05-08 08:00:00                0.81  
        2017-05-08 08:30:00                0.81  
        2017-05-08 09:00:00                0.81  
        2017-05-08 09:30:00                0.81  

Starting Training with Bolt model(s)


Chronos[amazon__chronos-bolt-small]/W0 ignores following hyperparameters: ['optimization.max_epochs']. See the documentation for Chronos[amazon__chronos-bolt-small]/W0 for the list of supported hyperparameters.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

...

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/478M [00:00<?, ?B/s]

	-1.8703       = Validation score (-MASE)
	6.89    s     = Training runtime
	0.51    s     = Validation (prediction) runtime
Training timeseries model Chronos2FineTuned. 


Step,Training Loss


KeyboardInterrupt: 

In [ ]:
!zip -r /content/autogluon_c2_results.zip /content/autogluon_chronos2_results
!zip -r /content/autogluon_c_results.zip /content/autogluon_chronos_pv_forecast

  adding: content/autogluon_chronos2_results/ (stored 0%)
  adding: content/autogluon_chronos2_results/utils/ (stored 0%)
  adding: content/autogluon_chronos2_results/utils/data/ (stored 0%)
  adding: content/autogluon_chronos2_results/utils/data/train.pkl (deflated 69%)
  adding: content/autogluon_chronos2_results/predictor.pkl (deflated 32%)
  adding: content/autogluon_chronos2_results/models/ (stored 0%)
  adding: content/autogluon_chronos2_results/models/Chronos[autogluon__chronos-bolt-small]/ (stored 0%)
  adding: content/autogluon_chronos2_results/models/Chronos[autogluon__chronos-bolt-small]/transformers_logs/ (stored 0%)
  adding: content/autogluon_chronos2_results/models/Chronos[autogluon__chronos-bolt-small]/transformers_logs/checkpoint-400/ (stored 0%)
  adding: content/autogluon_chronos2_results/models/Chronos[autogluon__chronos-bolt-small]/transformers_logs/checkpoint-400/model.safetensors (deflated 6%)
  adding: content/autogluon_chronos2_results/models/Chronos[autogluon_

In [ ]:
!unzip "autogluon_c2_results" -d "restored_right"

Archive:  autogluon_c2_results.zip
   creating: restored_right/content/autogluon_chronos2_results/
   creating: restored_right/content/autogluon_chronos2_results/utils/
   creating: restored_right/content/autogluon_chronos2_results/utils/data/
  inflating: restored_right/content/autogluon_chronos2_results/utils/data/train.pkl  
  inflating: restored_right/content/autogluon_chronos2_results/predictor.pkl  
   creating: restored_right/content/autogluon_chronos2_results/models/
   creating: restored_right/content/autogluon_chronos2_results/models/Chronos[autogluon__chronos-bolt-small]/
   creating: restored_right/content/autogluon_chronos2_results/models/Chronos[autogluon__chronos-bolt-small]/transformers_logs/
   creating: restored_right/content/autogluon_chronos2_results/models/Chronos[autogluon__chronos-bolt-small]/transformers_logs/checkpoint-400/
  inflating: restored_right/content/autogluon_chronos2_results/models/Chronos[autogluon__chronos-bolt-small]/transformers_logs/checkpoint-4